In [1]:
import time
from iot_mqtt import PumpMQTT, UltraMQTT, HeatMQTT, start_broker_if_needed, stop_broker
proc = start_broker_if_needed()  # only if you don’t run the Windows service

broker = "192.168.0.100"

pumps = PumpMQTT(broker=broker, username="pump1", password="pump",
                base_topic="pumps/01", client_id="pyctl-pumps")
ultra = UltraMQTT(broker=broker, username="ultra1", password="ultra",
                base_topic="ultra/01", client_id="pyctl-ultra")
heat = HeatMQTT(broker=broker, username="heat1", password="heat",
                base_topic="heat/01", client_id="pyctl-heat")

pumps.ensure_connected()
ultra.ensure_connected()
heat.ensure_connected()
time.sleep(1)  # wait for connections to settle
print("Connected to broker")

[broker] Logging to: C:\Users\13538\OneDrive\Desktop\Arduino\mosq-python.log
[broker] Starting Mosquitto (-v for logs)...
[broker] Broker is ready.
[pyctl-pumps] Connecting to 192.168.0.100:1883 (attempt 1)...
[pyctl-pumps] Connection failed; retrying in 0.6 seconds...
[pyctl-pumps] Connecting to 192.168.0.100:1883 (attempt 2)...


KeyboardInterrupt: 

In [ ]:
# Show live status from each node for a couple seconds
pumps.status(seconds=2.0)
ultra.status(seconds=2.0)
heat.status(seconds=2.0)

In [ ]:
# Pumps
pumps.on(1)                 # pump 1 ON
time.sleep(1)
pumps.on(1, 2000)           # pump 1 ON for 2 s (auto-off)
time.sleep(2.5)

In [ ]:
# Ultrasonic
ultra.on_for(2, 1500)       # ultrasonic ch2 ON for 1.5 s
time.sleep(2)

In [ ]:
# Heater + thermistor demo
heat.set_target(1, 42.0)         # set target to 42C (ESP retains on set/1)
heat.pid_on(1)                   # enable PID loop on ESP
# actively request a reading now:
try:
    for i in range(200):
        t = heat.get_base_temp(1, timeout_s=5.0)
        print("Temp(ch1) =", t)
        time.sleep(1)
except TimeoutError as e:
    print("Temp read timeout:", e)

heat.pid_off(1)                  # stop PID
heat.set_pwm(1, 0)               # ensure PWM is off
heat.off(1)                      # ensure relay is off

In [ ]:
# Cleanup
pumps.disconnect()
ultra.disconnect()
heat.disconnect()
stop_broker(proc)